In [72]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
import math
import sklearn.preprocessing as preprocessing
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('Indicators.csv')
data.shape

(5656458, 6)

In [11]:
rel_columns_7 = ['Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Age dependency ratio (% of working-age population)',
 'Age dependency ratio, young (% of working-age population)',
 'Birth rate, crude (per 1,000 people)',
 'Fertility rate, total (births per woman)',
 'Life expectancy at birth, female (years)',
 'Life expectancy at birth, male (years)',
 'Life expectancy at birth, total (years)',
 'Mortality rate, adult, female (per 1,000 female adults)',
 'Mortality rate, infant (per 1,000 live births)',
 'Mortality rate, under-5 (per 1,000)',
 'Population, ages 0-14 (% of total)',
 'Population, ages 15-64 (% of total)',
 'Survival to age 65, female (% of cohort)',
 'Survival to age 65, male (% of cohort)',
 'Adjusted net enrolment rate, primary, both sexes (%)',
 'Adjusted net enrolment rate, primary, female (%)',
 'Gross enrolment ratio, secondary, both sexes (%)',
 'Gross enrolment ratio, secondary, female (%)',
 'Gross enrolment ratio, secondary, male (%)',
 'Lower secondary completion rate, both sexes (%)',
 'Net enrolment rate, secondary, both sexes (%)',
 'Net enrolment rate, secondary, female (%)',
 'Net enrolment rate, secondary, male (%)',
 'Primary completion rate, both sexes (%)',
 'Primary completion rate, female (%)',
 'Survival rate to the last grade of primary education, both sexes (%)',
 'Lower secondary completion rate, female (%)',
 'Lower secondary completion rate, male (%)',
 'Survival rate to the last grade of primary education, female (%)',
 'Survival rate to the last grade of primary education, male (%)',
 'Survival rate to Grade 5 of primary education, female (%)',
 'Youth literacy rate, population 15-24 years, both sexes (%)',
 'Youth literacy rate, population 15-24 years, male (%)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Wanted fertility rate (births per woman)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Improved sanitation facilities (% of population with access)',
 'Improved sanitation facilities, rural (% of rural population with access)',
 'Improved sanitation facilities, urban (% of urban population with access)',
 'Improved water source (% of population with access)',
 'Improved water source, rural (% of rural population with access)',
 'Lifetime risk of maternal death (%)',
 'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
 'Mortality rate, infant, female (per 1,000 live births)',
 'Mortality rate, infant, male (per 1,000 live births)',
 'Mortality rate, neonatal (per 1,000 live births)',
 'Mortality rate, under-5, female (per 1,000 live births)',
 'Mortality rate, under-5, male (per 1,000 live births)',
 'Prevalence of anemia among children (% of children under 5)',
 'Prevalence of anemia among pregnant women (%)',
 'Renewable energy consumption (% of total final energy consumption)',
 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)',
 'Cause of death, by non-communicable diseases (% of total)',
 'Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)',
 'Survey mean consumption or income per capita, total population (2005 PPP $ per day)']

In [12]:
for i in rel_columns_7:
    if len(data[data['IndicatorName']==i]) < 500:
        rel_columns_7.remove(i)

len(rel_columns_7)

54

In [5]:
rel_columns_8 = ['Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Birth rate, crude (per 1,000 people)',
 'Net enrolment rate, secondary, both sexes (%)',
 'Net enrolment rate, secondary, female (%)',
 'Net enrolment rate, secondary, male (%)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Mortality rate, under-5, female (per 1,000 live births)',
 'Mortality rate, under-5, male (per 1,000 live births)',
 'Prevalence of anemia among children (% of children under 5)',
 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)',
 'Cause of death, by non-communicable diseases (% of total)',
 'Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)',
 'Survey mean consumption or income per capita, total population (2005 PPP $ per day)']

In [7]:
# 여기서부터 관련도 높은 feature 뽑는 것
target_id = 'Adolescent fertility rate (births per 1,000 women ages 15-19)';

target = data[data['IndicatorName']==target_id]

In [10]:
merged = data[data['IndicatorName']==target_id]

for i in rel_columns_8:
    target = data[data['IndicatorName']==i]
    target.rename(columns={"Value": i}, inplace = True)
    target=target.drop(columns=['IndicatorCode', 'IndicatorName', 'CountryName'])
#    print(target.head())
    merged_bef = merged
    merged = pd.merge(merged, target)
    if len(merged) < 100:
        merged = merged_bef
#    print(merged.head())

merged

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)","Mortality rate, under-5, female (per 1,000 live births)","Mortality rate, under-5, male (per 1,000 live births)",Prevalence of anemia among children (% of children under 5),"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by non-communicable diseases (% of total)"
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,53.829472,53.829472,28.282837,52.047924,48.331795,55.668274,56.807549,63.832238,45.362613,34.960297,56.536043
1,Caribbean small states,CSS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,71.619280,71.619280,20.044584,73.236755,75.206886,71.290482,23.874922,30.173376,36.028440,20.792559,70.364838
2,Central Europe and the Baltics,CEB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,23.946493,23.946493,9.770916,84.175941,85.047661,83.343834,12.595462,15.489390,26.704137,3.785028,89.350491
3,East Asia & Pacific (all income levels),EAS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,19.532642,19.532642,15.796701,56.540958,55.599720,57.432999,36.879240,42.919208,23.088350,16.192021,74.081354
4,East Asia & Pacific (developing only),EAP,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,20.675132,20.675132,16.400833,53.626995,52.593822,54.606003,39.200000,45.200000,23.981065,16.852771,73.273248
5,Euro area,EMU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,11.320513,11.320513,10.637990,85.865150,86.411331,85.345810,5.390092,6.583297,12.419406,4.871347,89.703603
6,Europe & Central Asia (all income levels),ECS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,25.716707,25.716707,11.668391,83.628906,83.499245,83.766197,20.220487,24.841956,23.552725,6.268130,85.551096
7,Europe & Central Asia (developing only),ECA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,38.296175,38.296175,15.354005,77.895233,76.662506,79.116798,38.100000,46.200000,35.447080,8.809273,82.919354
8,European Union,EUU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,16.191918,16.191918,10.593453,86.593323,87.267128,85.951340,6.734886,8.260032,14.629613,5.508877,89.138797
9,Fragile and conflict affected situations,FCS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,99.299216,99.299216,38.058768,27.228626,23.617943,30.775312,117.134118,131.036462,62.338067,63.153939,27.719711


In [13]:
merged = data[data['IndicatorName']==target_id]

for i in rel_columns_7:
    target = data[data['IndicatorName']==i]
    target.rename(columns={"Value": i}, inplace = True)
    target=target.drop(columns=['IndicatorCode', 'IndicatorName', 'CountryName'])
#    print(target.head())
    merged_bef = merged
    merged = pd.merge(merged, target)
    if len(merged) < 100:
        merged = merged_bef
#    print(merged.head())

merged

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, young (% of working-age population)","Birth rate, crude (per 1,000 people)",...,"Primary completion rate, both sexes (%)","Primary completion rate, female (%)","Survival rate to the last grade of primary education, both sexes (%)","Lower secondary completion rate, female (%)","Lower secondary completion rate, male (%)","Survival rate to the last grade of primary education, female (%)","Survival rate to the last grade of primary education, male (%)","Survival rate to Grade 5 of primary education, female (%)","Youth literacy rate, population 15-24 years, both sexes (%)","Youth literacy rate, population 15-24 years, male (%)"
0,Cameroon,CMR,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1976,192.6910,192.6910,90.853880,83.797430,45.286,...,49.683950,41.274080,43.800400,5.275850,12.108020,39.490910,47.466930,47.774690,68.817030,79.591650
1,Swaziland,SWZ,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1976,151.0790,151.0790,104.088943,98.525393,48.785,...,57.619080,58.914050,47.702160,24.173520,31.364390,51.056870,44.605740,62.883740,75.452450,76.083080
2,Peru,PER,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1981,77.8576,77.8576,84.187231,77.417421,35.347,...,79.187480,74.514520,58.814030,52.698380,60.267600,55.539250,61.969910,59.693780,93.119800,96.513340
3,Syrian Arab Republic,SYR,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1981,103.5042,103.5042,106.899479,100.839974,43.830,...,74.740050,59.972170,62.177800,36.053930,57.614490,53.790640,69.224740,65.495590,74.276410,88.831990
4,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,93.6652,93.6652,88.163978,83.176254,33.498,...,89.049120,96.942510,66.738180,79.125750,61.253530,72.842350,60.703690,80.663320,89.258300,85.900070
5,Chad,TCD,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,212.4100,212.4100,107.037747,101.110015,50.855,...,22.244770,12.728440,46.897480,3.759570,13.788120,40.461979,50.782871,49.275860,37.557354,55.686195
6,Estonia,EST,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,26.2586,26.2586,48.373608,26.087802,9.400,...,94.281700,93.927010,98.512337,91.776863,86.307358,97.925148,99.065331,98.285538,99.770500,99.729103
7,Ghana,GHA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,81.2662,81.2662,80.015715,74.580788,34.589,...,71.773308,67.285889,59.424042,49.203178,59.357590,57.971741,60.747662,65.174744,70.663063,75.945892
8,Lao PDR,LAO,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,79.7690,79.7690,88.255687,81.544676,31.910,...,67.148918,61.322262,53.478180,29.617189,40.776279,54.085419,52.953281,54.085419,80.598557,88.061966
9,Lesotho,LSO,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,91.3626,91.3626,84.311126,75.983478,31.443,...,63.688278,74.731018,55.029980,31.785629,23.292999,62.900612,47.513359,74.300583,90.926903,84.825584


In [74]:
# 기준: value 등급? 만듦
print(min(merged['Value']))
print(max(merged['Value']))
print(merged['Value'].mean())

6.8839999999999995
218.2176
76.98698019801981


In [78]:
merged['Value']/100

0      1.926910
1      1.510790
2      0.778576
3      1.035042
4      0.936652
         ...   
96     1.006240
97     0.773250
98     0.840360
99     0.659700
100    0.408240
Name: Value, Length: 101, dtype: float64

In [ ]:
for i in range(len(merged)):
    print(sigmoid(merged.iloc[i]['Value']))
    
merged

In [51]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [111]:
merged['grade'] = merged['Value']/50
for i in range(len(merged)):
    merged.loc[i, 'grade'] = math.ceil(merged.loc[i, 'grade'])
merged

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, young (% of working-age population)","Birth rate, crude (per 1,000 people)",...,"Primary completion rate, female (%)","Survival rate to the last grade of primary education, both sexes (%)","Lower secondary completion rate, female (%)","Lower secondary completion rate, male (%)","Survival rate to the last grade of primary education, female (%)","Survival rate to the last grade of primary education, male (%)","Survival rate to Grade 5 of primary education, female (%)","Youth literacy rate, population 15-24 years, both sexes (%)","Youth literacy rate, population 15-24 years, male (%)",grade
0,Cameroon,CMR,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1976,192.6910,192.6910,90.853880,83.797430,45.286,...,41.274080,43.800400,5.275850,12.108020,39.490910,47.466930,47.774690,68.817030,79.591650,4.0
1,Swaziland,SWZ,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1976,151.0790,151.0790,104.088943,98.525393,48.785,...,58.914050,47.702160,24.173520,31.364390,51.056870,44.605740,62.883740,75.452450,76.083080,4.0
2,Peru,PER,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1981,77.8576,77.8576,84.187231,77.417421,35.347,...,74.514520,58.814030,52.698380,60.267600,55.539250,61.969910,59.693780,93.119800,96.513340,2.0
3,Syrian Arab Republic,SYR,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1981,103.5042,103.5042,106.899479,100.839974,43.830,...,59.972170,62.177800,36.053930,57.614490,53.790640,69.224740,65.495590,74.276410,88.831990,3.0
4,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,93.6652,93.6652,88.163978,83.176254,33.498,...,96.942510,66.738180,79.125750,61.253530,72.842350,60.703690,80.663320,89.258300,85.900070,2.0
5,Chad,TCD,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,212.4100,212.4100,107.037747,101.110015,50.855,...,12.728440,46.897480,3.759570,13.788120,40.461979,50.782871,49.275860,37.557354,55.686195,5.0
6,Estonia,EST,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,26.2586,26.2586,48.373608,26.087802,9.400,...,93.927010,98.512337,91.776863,86.307358,97.925148,99.065331,98.285538,99.770500,99.729103,1.0
7,Ghana,GHA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,81.2662,81.2662,80.015715,74.580788,34.589,...,67.285889,59.424042,49.203178,59.357590,57.971741,60.747662,65.174744,70.663063,75.945892,2.0
8,Lao PDR,LAO,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,79.7690,79.7690,88.255687,81.544676,31.910,...,61.322262,53.478180,29.617189,40.776279,54.085419,52.953281,54.085419,80.598557,88.061966,2.0
9,Lesotho,LSO,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2000,91.3626,91.3626,84.311126,75.983478,31.443,...,74.731018,55.029980,31.785629,23.292999,62.900612,47.513359,74.300583,90.926903,84.825584,2.0


In [117]:
merged['grade'].unique()

array([4., 2., 3., 5., 1.])

In [116]:
merged['grade'].value_counts()

2.0    61
1.0    21
3.0    11
4.0     6
5.0     2
Name: grade, dtype: int64

In [114]:
print(min(merged['grade']))
print(max(merged['grade']))
print(merged['grade'].mean())

1.0
5.0
2.0792079207920793


In [14]:
# scaling data (MinMaxScaler)
def minmax(X):
    scaler = preprocessing.MinMaxScaler()
    fit = scaler.fit(X)

    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=list(X.index.values))
    return X_scaled

In [15]:
# scaling data (MinMaxScaler)
def standard(X):
    scaler = preprocessing.StandardScaler()
    fit = scaler.fit(X)

    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=list(X.index.values))
    return X_scaled


In [16]:
# cleansing dataset
def cleansing(X):
    # cleansing missing data
    X.dropna(axis=0, thresh=1, inplace=True)
    # cleansing outliers
    robust = preprocessing.RobustScaler().fit(X)

    X_robust = robust.transform(X)
    X_robust = pd.DataFrame(X, columns=X.columns, index=list(X.index.values))
    return X_robust

In [17]:
eps=[0.5 ]
min_samples=[  100 ]
dis_measure = ['euclidean', 'hamming']

In [18]:
def purity_score(y, y_label):
    print(y)
    print(y_label)
    unique_labels=set(y_label)
    
    sum_=0
    
    purity = pd.DataFrame(0, index=np.arange(len(unique_labels)), columns=["0", "1"])
    print(purity)
    for i in range(len(y_label)):
        if y[i] ==0:
            purity.iloc[y_label[i], 0] = purity.iloc[y_label[i], 0]+1
        else:
            purity.iloc[y_label[i], 1] = purity.iloc[y_label[i], 1]+1

    for j in range(len(purity)):
        if purity.iloc[j, 0] > purity.iloc[j, 1]:
            sum_=sum_ + purity.iloc[j, 0]
        else:
            sum_=sum_ + purity.iloc[j, 1]
    
    return sum_/len(y)

In [19]:
def none(i, j, n):
    X=df.drop(columns=['CountryName', 'CountryCode', 'IndicatorName', 'IndicatorCode', 'Year', 'Value'])
    y=df['class']
    X = cleansing(X)
    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [20]:
def labeling(i, j, n):
    labelEncode()
    X=df.drop(columns=['class'])
    y=df['class']

    X = cleansing(X)

    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [21]:
def label_minmax(i, j, n):
    labelEncode()
    X=df.drop(columns=['class'])
    y=df['class']

    X_scaled = minmax(X)
    X_scaled = cleansing(X_scaled)
    
    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X_scaled) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [22]:
def label_standard(i, j, n):
    labelEncode()
    X=df.drop(columns=['class'])
    y=df['class']

    X_scaled = standard(X)
    X_scaled = cleansing(X_scaled)
    
    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X_scaled) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [ ]:
for i in eps:
    for j in min_samples:
        for n in dis_measure:
            print(i, j, n)
            # varying preprocessing            

            # none
            if n == 'hamming':
                print('none')
                none(i, j, n)
            
            # labelEncoding
            print('\nlabelEncoding')
            labeling(i, j, n)
            
            # labelEncoding + MinMaxScaler
            print('\nlabelEncoding + MinMaxScaler')
            label_minmax(i, j, n)
            
            # labelEncoding + StandardScaler
            print('\nlabelEncoding + StandardScaler')
            label_standard(i, j, n)
            print('\n')
